In [ ]:
# Imports 
import numpy as np
import torchvision.transforms as transforms
import torch
from glob import glob
from torchvision import datasets


ModuleNotFoundError: No module named 'torch'

In [86]:
# Load data
data_dir = 'flower_data'
train_dir = data_dir + '/train/*/*'
valid_dir = data_dir + '/valid/*/*'

# load filenames for flowers images
flowers_train = np.array(glob(train_dir))
flowers_valid = np.array(glob(valid_dir))

# print number of images in each dataset
print('There are %d total flowers train images.' % len(flowers_train))
print('There are %d total flowers valid images.' % len(flowers_valid))


There are 6552 total flowers train images.
There are 818 total flowers valid images.


In [84]:
# TODO: Define your transforms for the training and validation sets
data_transforms = transforms.ToTensor()

# TODO: Load the datasets with ImageFolder
image_datasets = datasets.MNIST(root='ImageFolder', train=True, download=True, transform=data_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size, num_workers=num_workers)


NameError: name 'transforms' is not defined

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [ ]:


# TODO: Build and train your network

In [ ]:
# TODO: Save the checkpoint

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the mo

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

In [ ]:
# TODO: Display an image along with the top 5 classes